# Tron x Snake

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))

Ce notebook sert de démonstration et d'explication de l'implémentation d'une variante de [Snake](https://en.wikipedia.org/wiki/Snake_(video_game_genre)), inspirée du jeu d'arcade [Tron light cycles](https://en.wikipedia.org/wiki/Tron_(video_game)).

In [2]:
from itertools import chain

from front import SnakeGameWindow
from agent import PlayerSnakeAgent, AStarSnakeAgent, AStarOffensiveSnakeAgent
from world import SnakeWorld, EuclidianDistanceHeuristic, ManhattanDistanceHeuristic

Le programme est structuré comme un système multi-agent:
- Le monde est une grille de case dans laquelle apparaît de la nourriture à des positions aléatoires.
- Les agents sont des serpents se déplaçant dans le monde dans le but d'être le dernier survivant.

<div style="text-align: center;">
    <img src="inclusion_map.png" width="250">
    <p><em>Structure du code, généré avec inclusionmap (pip install inclusionmap)</em></p>
</div>

Les règles permettant de simuler l'évolution du monde lors d'une partie sont les suivantes:
- Quand un serpent mange une nourriture, il s'allonge d'une case.
- Quand la tête d'un serpent mord sa propre queue, le serpent se sectionne et diminue de taille.
- Quand la tête d'un serpent A mord la queue d'un serpent B, le serpent A meurt.

Les agents peuvent:
- Être controllé par un joueur avec les quatre flèches directionnelles du clavier (`PlayerSnakeAgent`).
- Être controllé par un programme (`AStarSnakeAgent` ou `AStarOffensiveSnakeAgent`).

## Création d'un adversaire passif

La classe `AStarSnakeAgent` implémente un adversaire passif: un agent dont l'objectif est de se nourrir pour grandir le plus vite possible, tout en évitant les obstacles.

L'agent suit le chemin le plus court vers la nouriture la plus proche, en évitant les cases occupées par:
- sa propre queue,
- les corps des autres agents (dont le joueur).

Cependant, à chaque étape du jeu, la position des obstacles dans le monde évolue car tous les agents se déplacent.
Pour s'adapter à ces changements, `AStarSnakeAgent` recalcule dynamiquement son chemin à chaque étape grâce à l'algorithme A*.

La cellule ci-dessous permet au joueur de controller le serpent bleu avec les flèches directionnelles pour affronter un serpent jaune qui est une instance de `AStarSnakeAgent`.
On peut constater que l'adversaire ne cherche jamais à attaquer le joueur mais se dirige simplement vers la nourriture en l'évitant.
Le joueur peut survivre longtemps sans jamais changer de direction, ce qui rend le jeu ennuyant. 

In [3]:
width = 21
height = 21

world = SnakeWorld(width, height, n_food=1)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
)

player_agents = [blue]
ai_agents = [yellow]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    ui_size_coeff=20
)
gui.title("Adversaire passif")
gui.mainloop()

## Création d'un adversaire offensif pour rendre le jeu plus intéressant

La classe `AStarOffensiveSnakeAgent` implémente un adversaire offensif.
C'est un agent qui essaie de placer sa queue devant la tête de sa cible pour la tuer.
Si l'agent ne trouve pas de chemin lui permettant d'attaquer sa cible, alors il se dirige vers la nouriture la plus proche, à la manière de `AStarSnakeAgent`.

Pour chercher un chemin d'attaque, l'agent suppose que sa cible ne va pas changer de direction lors des prochaines étapes du jeu.
Les cases qui sont en ligne droite devant la tête de la cible sont alors des points d'impacts potentiels.

L'attaquant calcule grâce à l'algorithme A* les chemins les plus courts allant de sa tête aux points d'impacts potentiels.
Étant donné un de ces chemin `path`, on définit les grandeurs suivantes:
- `len(path)` est le nombre d'étapes nécessaires pour que l'attaquant atteigne le point d'impact en suivant le chemin `path` qu'il a calculé avec A*.
- `len(agent)` est la longueur du corps de l'attaquant.
- `impact_delay` est la distance en ligne droite séparant la tête de la cible et le point d'impact. Il s'agit du nombre d'étapes nécessaires pour que la cible arrive au point d'impact si elle ne change jamais de direction.

`advance = impact_delay - len(path)` est le nombre d'étapes d'avance que l'attaquant a sur sa cible pour atteindre le point d'impact.
L'attaquant doit arriver au point d'impact strictement avant sa cible donc cela impose que `advance > 0`.
Cependant, pour que la cible morde effectivement la queue de l'attaquant, celui-ci ne doit pas arriver trop en avance au point d'impact.
Cela impose que `advance < len(agent)`.

Le chemin `path` est donc une attaque valide si et seulement si `0 < advance < len(agent)`, c.a.d si la longueur du chemin `path` est strictement comprise entre `impact_delay - len(agent)` et `impact_delay`.

Parmis toutes les attaques valides, `AStarOffensiveSnakeAgent` choisit celle qui:
1) minimise `impact_delay`: pour que l'attaque surprenne le plus possible la cible,
2) minimise `len(path)`: pour que l'attaquant ai le moins de distance à couvrir afin d'effectuer son attaque.

Cet algorithme de recherche est implémenté dans la méthode `compute_attack_path` de la classe `AStarOffensiveSnakeAgent` dans le fichier `agent.py`.

La cellule ci-dessous permet au joueur de controller le serpent bleu avec les flèches directionnelles pour affronter un serpent jaune qui est une instance de `AStarOffensiveSnakeAgent`.
Pour survivre longtemps, le joueur est forcé d'éviter les attaques de son adversaire en changeant régulièrement de direction: le jeu est maintenant divertissant !

In [ ]:
width = 21
height = 21

world = SnakeWorld(width, height, n_food=1)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
)

yellow.add_opponent(blue)

player_agents = [blue]
ai_agents = [yellow]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    ui_size_coeff=20
)
gui.title("Adversaire offensif")
gui.mainloop()

## Paramétrer le comportement des IA

Le comportement des agents IA peut être ajusté à l'aide des deux paramètres `latency` et `caution`.
Il devient alors possible de:
- Dégrader les performances d'un agent pour que le joueur soit capable de l'attaquer: `latency` est le nombre d'étapes de jeu que l'agent doit attendre avant d'être capable de recalculer son chemin pour s'adapter aux modifications du monde.
- Rendre une IA plus prudente: lorsqu'il calcule un chemin vers la nourriture la plus proche, un agent évite non seulement sa propre queue et les corps des autres agents, mais aussi les cases situées dans un rayon de `caution` autour de leurs têtes.

La cellule ci-dessous permet au joueur de controller le serpent bleu avec les flèches directionnelles pour affronter d'autres serpents dont les comportements sont variés.

- Le joueur peut essayer de tuer les serpents violets et verts qui sont moins réactifs que le serpent jaune.
- Le serpent jaune est très aggressif: il essaie de tuer le joueur et le serpent violet.
- Le serpent violet est moins aggressif: il n'essaie de tuer que le joueur lorsque l'occasion se présente.
- Le serpent vert est prudent et craintif: il n'essaie que de se nourir.

In [ ]:
width = 30
height = 30
world = SnakeWorld(width, height, n_food=2)

blue = PlayerSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1)
)

yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)

purple = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=3,
    caution=1
)

green = AStarSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=3,
    caution=3
)

yellow.add_opponent(purple)
yellow.add_opponent(blue)
purple.add_opponent(blue)

player_agents = [blue]
ai_agents = [yellow, purple, green]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindow(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=100,
    explain_ai=True,
    ui_size_coeff=20
)
gui.title("Comportement paramétrable")
gui.mainloop()

## Combat de programmes

Avec la musique de TRON, c'est mieux !

In [ ]:
try:
    import pygame
    class SnakeGameWindowWithCoolMusic(SnakeGameWindow):
        def mainloop(self, n = 0):
            pygame.mixer.init()
            pygame.mixer.music.load("music/tron.mp3")
            pygame.mixer.music.play(-1)
            pygame.mixer.music.set_pos(9.7)
            return super().mainloop(n)

        def destroy(self):
            pygame.mixer.music.stop()
            return super().destroy()
except ModuleNotFoundError:
    SnakeGameWindowWithCoolMusic = SnakeGameWindow

Enjoy le scénario:
- Les jaunes attaquent les bleus
- Les bleus attaquent les rouges/violets
- Les rouges/violets attaquent les verts
- Les verts attaquent les jaunes

In [ ]:
width = 45
height = 45
world = SnakeWorld(width, height, n_food=3)

blue = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=0
)

dark_blue = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(9, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=0
)


yellow = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)

orange = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-10, y) for y in range(10, 1, -1)],
    initial_dir=(0, 1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=1
)


purple = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(4, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=4,
    caution=4
)

red = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(9, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=4,
    caution=4
)


green = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-5, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=3
)

light_green = AStarOffensiveSnakeAgent(
    world,
    initial_pos=[(width-10, y) for y in range(height-10, height-1, 1)],
    initial_dir=(0, -1),
    heuristic_type=EuclidianDistanceHeuristic,
    latency=0,
    caution=3
)

blue.add_opponent(purple)
blue.add_opponent(red)
dark_blue.add_opponent(purple)
dark_blue.add_opponent(red)

purple.add_opponent(green)
purple.add_opponent(light_green)
red.add_opponent(green)
red.add_opponent(light_green)

green.add_opponent(yellow)
green.add_opponent(orange)
light_green.add_opponent(yellow)
light_green.add_opponent(orange)

yellow.add_opponent(blue)
yellow.add_opponent(dark_blue)
orange.add_opponent(blue)
orange.add_opponent(dark_blue)


player_agents = []
ai_agents = [blue, yellow, purple, green, red, orange, light_green, dark_blue]

for agent in chain(player_agents, ai_agents):
    world.attach_agent(agent)

gui = SnakeGameWindowWithCoolMusic(
    world,
    player_agents=player_agents,
    ai_agents=ai_agents,
    time_step=50,
    explain_ai=True,
    ui_size_coeff=17
)
gui.title("Combat de programmes")
gui.mainloop()